In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [22]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [33]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if i%log_interval==0:
            print('Train Epoch: {}/{} \tLoss: {:.6f}'.format(i*len(data), len(train_loader.dataset), loss.item()))

In [43]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss+=F.nll_loss(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct+=pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(test_loader.dataset)
    print('Test set: Average Loss: {:.4f}, Accuracy: {}'.format(test_loss, 100.*correct/len(test_loader.dataset)))

In [44]:
batch_size = 64
test_batch_size = 128
epochs = 10
lr = 0.01
log_interval = 100

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
train_transforms = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])

In [47]:
torch.manual_seed(42)

In [48]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=train_transforms), batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=train_transforms), batch_size=test_batch_size, shuffle=True
)

In [49]:
model = Net().to(device)

In [51]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.5)

In [52]:
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 0/60000 	Loss: 2.312211
Train Epoch: 6400/60000 	Loss: 1.897876
Train Epoch: 12800/60000 	Loss: 0.878753
Train Epoch: 19200/60000 	Loss: 0.580373
Train Epoch: 25600/60000 	Loss: 0.645617
Train Epoch: 32000/60000 	Loss: 0.545144
Train Epoch: 38400/60000 	Loss: 0.468785
Train Epoch: 44800/60000 	Loss: 0.513736
Train Epoch: 51200/60000 	Loss: 0.315046
Train Epoch: 57600/60000 	Loss: 0.359710
Test set: Average Loss: 0.0012, Accuracy: 95.39
Train Epoch: 0/60000 	Loss: 0.220207
Train Epoch: 6400/60000 	Loss: 0.269178
Train Epoch: 12800/60000 	Loss: 0.406765
Train Epoch: 19200/60000 	Loss: 0.270464
Train Epoch: 25600/60000 	Loss: 0.253265
Train Epoch: 32000/60000 	Loss: 0.332967
Train Epoch: 38400/60000 	Loss: 0.634007
Train Epoch: 44800/60000 	Loss: 0.189499
Train Epoch: 51200/60000 	Loss: 0.323913
Train Epoch: 57600/60000 	Loss: 0.253260
Test set: Average Loss: 0.0008, Accuracy: 96.83
Train Epoch: 0/60000 	Loss: 0.273724
Train Epoch: 6400/60000 	Loss: 0.331078
Train Epoch: 1280